# Diplomatura en ciencia de datos, aprendizaje automático y sus aplicaciones - Edición 2023 - FAMAF (UNC)

## Análisis y visualización de datos

### Trabajo práctico entregable - Grupo 22 - Parte 2

**Integrantes:**
- Chevallier-Boutell, Ignacio José
- Ribetto, Federico Daniel
- Rosa, Santiago

**Seguimiento:** Meinardi, Vanesa

---

## Librerías

In [ ]:
import io
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy
from scipy.stats import ttest_ind as Test_t
from statsmodels.stats.power import tt_ind_solve_power as Power

pd.set_option('display.max_rows', 1000) # cambiar el número de filas que se mostrarán usando display.max_rows.
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.options.mode.chained_assignment = None  # default='warn'

sns.set_context('talk')
sns.set_theme(style='white')

## Lectura del dataset

El dataset a utilizar es la encuesta Sysarmy del año 2022 versión 2, en formato csv, la cual es una una encuesta personal y voluntaria que busca relevar información sobre salarios y condiciones de trabajo de programadores, que se realiza anualmente. Se analizarán sólo los datos provenientes de Argentina. 

Se utilizará un dataset que ya ha tenido un pretratamiento: 
* Se eliminó el encabezado de la encuesta original.
* Se renombraron las columnas.

Este dataset está disponible en internet, desde donde lo usaremos.

In [ ]:
url = 'https://raw.githubusercontent.com/DiploDatos/AnalisisyVisualizacion/master/sysarmy_survey_2022_processed.csv'
df = pd.read_csv(url)
total_ans = len(df) # cantidad de respuestas en tel dataset

---
# Ejercicio 1 - Estimación

## Creación del DataFrame a utilizar

Vamos a basarnos en el mismo DataFrame que utilizamos para la parte 1 del entregable. Sus características son:
- No contiene valores faltantes.
- Contiene columnas renombradas para facilitar su manipulación.
- Está filtrada considerando un sueldo bruto estrictamente mayor al sueldo neto.
- Está filtrada considerando un sueldo neto entre un Salario Mínimo Vital y Móvil (SMVM) de \$ 38.940 (Abril de 2022) y 2 millones de pesos.
- Contiene el 85 % central de la distribución.
- El género está recategorizado dentro de varón cis y otros géneros.

In [ ]:
print(f'Cantidad inicial de filas: {total_ans}.')

# Selección de columnas
relevant_columns = ["salary_monthly_BRUTO",
                    "salary_monthly_NETO",
                    "profile_gender"]

df1 = df[relevant_columns]

# Eliminación de missing values
df1 = df1.dropna(subset=relevant_columns)

# Modificación de etiquetas a usar
df1.rename(columns = {"salary_monthly_BRUTO":'bruto', 
                      "salary_monthly_NETO":'neto',
                      "profile_gender":'genero'}
                      , inplace = True)

# Bruto mayor al neto
df1 = df1[df1["bruto"]>df1["neto"]]

# Entre un SMVM y 2 millones
SMVM = 38940
df1 = df1[df1["neto"]>=SMVM]
df1 = df1[df1["neto"]<=2*1e6]

# Dejar de lado la variable bruto
relevant_columns = ["neto", "genero"]
df1 = df1[relevant_columns]

# Tomar el 85% central de la distribución
k = 15
percentile_inf = df1["neto"].quantile(k * 0.5 / 100)
percentile_sup = df1["neto"].quantile((100 - k * 0.5) / 100)

df1 = df1[df1["neto"] > percentile_inf]
df1 = df1[df1["neto"] < percentile_sup]

# Recategorizar la variable género
df1.loc[:,'genero'] = df1.genero.replace(
    {'Varón Cis': 'varon_cis',
     'Varón cis': 'varon_cis',
     'Mujer': 'otros',
     'Mujer Cis': 'otros',
     'Mujer cis': 'otros',
     'Femenino': 'otros',
     'mujer': 'otros',
     'Mujer':'otros',
     'Queer':'otros',
     'Varón Trans':'otros',
     'No binarie':'otros',
     'Mujer Trans':'otros',
     'Fluido':'otros',
     'Bigénero':'otros',
     'Gay':'otros'
    })

print(f'Quedan {len(df1)} filas.')

display(df1[['genero', 'neto']].describe().T)
display(df1[['genero', 'neto']].groupby('genero').describe())

## Definiciones para la estadística

Vamos a separar nuestra población original en dos poblaciones: la población P1 de varones cis y la población P2 de otros géneros. Asumimos que ambas poblaciones distribuyen normalmente.
$$
P1 \sim \mathcal{N} \left( \mu_1, \sigma^2_1 \right) \quad ; \quad P2 \sim \mathcal{N} \left( \mu_2, \sigma^2_2 \right)
$$


De dicha población original tenemos una muestra de $n=3481$ elementos: $n_1=2845$ constituyen la muestra M1 de la población P1, mientras que los otros $n_2=636$ constituyen la muestra M2 de la población P2. Notamos que ambas muestras son grandes ($n_1, n_2 > 30$) e independientes. A cada muestra le calculamos su media, su desviación estándar y su varianza:
$$
\overline{m}_1 = \$\ 227.844 \quad ; \quad s_1 = \$\ 88.618 \quad ; \quad s_1^2 = \$^2\ 7.853.228.914
$$

$$
\overline{m}_2 = \$\ 203.765 \quad ; \quad s_2 = \$\ 80.404 \quad ; \quad s_2^2 = \$^2\ 6.464.777.223
$$

Nuestro objetivo es utilizar la información de las muestras para:
- Hacer una estimación puntual sobre la diferencia entre las medias poblacionales.
- Hacer una estimación por intervalos sobre la diferencia entre las medias poblacionales para determinar un intervalo de confianza de nivel (1-$\alpha$).

In [ ]:
# Separación de la muestra según sea o no varón cis.
man = df1.genero == 'varon_cis'
M1 = df1[man].neto
n1 = len(M1)
m1 = M1.mean()
s1 = M1.std()

print('Descripción estadística de la muestra M1:')
print(f'\t > Tiene {n1} elementos.')
print(f'\t > Tiene una media igual a $ {m1:.0f}.')
print(f'\t > Tiene una desviación estándar igual a $ {s1:.0f}.')
print(f'\t > Tiene una varianza igual a {s1**2:.0f} pesos cuadrados.')
print('-------------------------------------------------------------------------')

M2 = df1[~man].neto
n2 = len(M2)
m2 = M2.mean()
s2 = M2.std()

print('Descripción estadística de la muestra M2:')
print(f'\t > Tiene {n2} elementos.')
print(f'\t > Tiene una media igual a $ {m2:.0f}.')
print(f'\t > Tiene una desviación estándar igual a $ {s2:.0f}.')
print(f'\t > Tiene una varianza igual a {s2**2:.0f} pesos cuadrados.')

## Estimación puntual

Como la media muestral $\overline{m}_1$ de la muestra M1 es un buen estimador de la media poblacional $\mu_1$ de la población P1 y la media muestral $\overline{m}_2$ de la muestra M2 es un buen estimador de la media poblacional $\mu_2$ de la población P2, resulta razonable que para estimar la diferencia $\mu  = \mu_1 - \mu_2$ recurramos al estimador $\overline{m} = \overline{m}_1 - \overline{m}_2$. Se sigue entonces que la estimación puntual buscada es
$$
\mu \sim \overline{m} = \$\ 24.079
$$

Esto quiere decir que el salario neto promedio de los varones cis es $ 24.079 mayor respecto al salario neto promedio de los otros géneros.

In [ ]:
# Diferencia de medias
m = m1 - m2
m

## Estimación por intervalos

Dado que ambas muestras son grandes, las varianzas de ambas poblaciones son desconocidas y no tenemos razones para asumir que son iguales, el pivote estadístico a utilizar será
$$
S = \sqrt{\dfrac{s_1^2}{n_1} + \dfrac{s_2^2}{n_2}} \Rightarrow Z = \dfrac{\overline{m} - \mu}{S} \stackrel{Asint}{\sim} \mathcal{N} \left( 0, 1 \right)
$$

A partir de esto, se puede ver que el invervalo de confianza para $\mu$ a un nivel de confianza (1-$\alpha$) queda determinado por
$$
\overline{m} \pm z_{\alpha/2} S
$$

donde $z_{\alpha/2}$ es el $\mathcal{Z}$-score asociado, el cual es bilateral. Tomando un nivel de significancia $\alpha=0.05$, tenemos $z_{0.025} = 1.96$.

Vemos entonces que el intervalo de confianza buscado es
$$
\left[\ \$\ 17.033\  ;\ \$\ 31.126\  \right]
$$

Considerando que la incertidumbre debería tener una única cifra significativa, tendríamos $\$\ 7046 \Rightarrow \$\ 7 \times 10^3$, con lo cual las estimaciones nos quedarían:
$$
\left[\ \$\ 17\  ;\ \$\ 31\  \right] \times 10^3 \Rightarrow \$\ \left( 24 \pm 7 \right) \times 10^3 
$$

En otros términos, podemos decir que estamos un 95% seguros de que la diferencia entre las medias poblacionales cae en el intervalo $\left[\ \$\ 17.000\  ;\ \$\ 31.000\  \right]$, *i.e.* estamos un 95% seguros de que el salario neto promedio de los varones cis es entre \$ 17.000 y \$ 31.000 mayor que el salario neto promedio de otros géneros. Porcentualmente, el salario neto promedio de los varones cis es entre un 8 % y un 15 % mayor a los otros géneros.

In [ ]:
S = np.sqrt((s1**2/n1) + (s2**2/n2))
z_0025 = 1.96

pm = z_0025 * S
print(f'Incertidumbre: $ {pm:.0f}')
LI = m - pm
LS = m + pm

print('Intervalo de confianza:')
print(f'\t > Límite inferior: $ {LI:.0f}')
print(f'\t > Límite superior: $ {LS:.0f}')

porc_inf = (m2 + LI.round(-3)) * 100 / m2 - 100
print(f'Porcentaje inferior: {porc_inf:.0f} %')
porc_sup = (m2 + LS.round(-3)) * 100 / m2 - 100
print(f'Porcentaje superior: {porc_sup:.0f} %')

## Relación entre el IC y test de hipótesis

Considerando un test de hipótesis, el intervalo de confianza dado por $(-z_{\alpha/2}\ ;\ z_{\alpha/2})$ representa la región de **no** rechazo de la hipótesis nula. En el siguiente ejercicio formalizaremos esto llevando a cabo dicho test de hipótesis y analizando sus resultados.

---
# Ejercicio 2 - Test de hipótesis

## Formalización

Planteamos las siguientes hipótesis:
- **Hipótesis nula:** el salario neto medio de varones cis es igual al salaro neto medio de otros géneros.
$$
H_0: \mu = 0
$$
- **Hipótesis alternativa:** los varones cis no tienen un salario neto promedio igual a aquel de los otros géneros, considerando un nivel de significancia del 5%.
$$
H_1: \mu \neq 0 \ @\  \alpha=0.05
$$

Como la prueba es bilateral, la región crítica es $(-\infty\ ;\ - 1.96] \cup [1.96\ ;\ \infty)$. El estadístico de prueba a utilizar para llevar a cabo el test de hipótesis es idéntico al pivote estadístico antes planteado, tomando el $\mu$ considerado en nuestra $H_0$.
$$
S = \sqrt{\dfrac{s_1^2}{n_1} + \dfrac{s_2^2}{n_2}} \Rightarrow Z = \dfrac{\overline{m}}{S} \stackrel{Asint}{\sim} \mathcal{N} \left( 0, 1 \right)
$$

Desde una perspectiva de valores críticos, como $Z_{normal}=6.6978$, el estadístico cae en la región de rechazo, lo cual nos lleva a rechazar $H_0$: con un nivel de significancia del 5%, los datos proporcionan evidencia suficiente para concluir que los salarios netos promedios entre varones cis y otros géneros **no** son iguales.

In [ ]:
Z_nml = m/S
Z_nml

## Enfoque desde el $p$-valor

Ya tenemos nuestras hipótesis planteadas y sabemos que nuestro estadístico vale $Z=6.6978$. De tablas sabemos que para $z=4.09$ el área truncada sobre la cola derecha de la distribución de es de 0.99998. El $p$-valor asociado a esto es $p_z = 1 - 0.99998 = 0.00002$. Con esto vemos que
$$
Z=6.6978 > z = 4.09 \Rightarrow p < p_z = 0.00002 \Rightarrow p < p_z < \alpha = 0.05
$$

Luego, rechazamos $H_0$: los datos proveen nuevamente evidencia suficiente de que, con un nivel de significancia del 5%, los salarios netos promedios entre varones cis y otros géneros **no** son iguales.

Para obtener un valor más preciso del $p$-valor, recurrimos a la función `scipy.stats.ttest_ind`. Dado que tenemos varianzas diferentes, debemos usar el test de Welch, el cual se activa usando el flag `equal_var=False`. Vemos que el estadístico obtenido mediante esta función es muy parecido al calculado previamente en el ejercicio anterior.
$$
Z_{Welch} \simeq Z_{normal}
$$

Además, corroboramos que $p < p_z$ ya que $p=4 \times 10^{-11}$.

In [ ]:
Z_welch, p_welch = Test_t(M1, M2, equal_var=False)
print(f'Estadístico de Welch: {Z_welch:.4f}')
print(f'p-valor de Welch: {p_welch}')

## Potencia de la prueba

---

La potencia (o poder estadístico) de una prueba de hipótesis es la probabilidad de que la prueba rechace correctamente la hipótesis nula $H_0$ cuando la hipótesis alternativa $H_1$ es verdadera, *i.e.* es la probabilidad de obtener un verdadero positivo. En otras palabras, la potencia nos dice cuántas chances tengo de detectar un efecto si es que verdaderamente existe. El poder estadístico está afectado por qué tanto se solapan las distribuciones nula y alternativa: se tendrá menor poder estadístico cuanto mayor sea superposición.

En estos casos, el nivel de significancia $\alpha$ nos permite controlar el error de tipo I (falso positivo), mientras que la tasa de falsos negativos $\beta$ cuantifica la probabilidad de cometer errores de tipo II (falso negativo).
        $$
        P(\text{rechazo $H_0$} | \text{$H_0$ verdadera}) = \alpha \quad ; \quad P(\text{no rechazo $H_0$} | \text{$H_0$ falsa}) = \beta
        $$
        $$
        P(\text{no rechazo $H_0$} | \text{$H_0$ verdadera}) = 1 - \alpha \quad ; \quad P(\text{rechazo $H_0$} | \text{$H_0$ falsa}) = 1 - \beta
        $$

Suele ser una función de las distribuciones posibles, determinada por algún parámetro bajo $H_1$. A mayor potencia, menor es la probabilidad de incurrir en un error de tipo II.
        $$
        \text{Potencia} = 1 - \beta
        $$

A partir de usar la función `tt_ind_solve_power`, obtenemos los resultados de la siguiente tabla:

| Potencia | $\beta$ | $n_{1,min}$  | $n_{2,min}$ |
| :----: | :---: | :-------: | :-------: |
| 0.8      | 0.2     | 481         |  107    |
| 0.9      | 0.1   |   643       |   144  |
| 0.95      | 0.05      | 795     |  178  |

donde $n_{1,min}$ y $n_{2,min}$ representan el tamaño mínimo que deberían tener las muestras M1 (varones cis) y M2 (otros géneros) para asegurar las potencias indicadas del test, respectivamente. Dado que $n_1=2845$ y $n_2=636$, podemos asegurar que las muestras eran lo suficientemente grande como para ser representativas de la tendencia general.

In [ ]:
effect_size = (m1 - m2) / s2
alpha = 0.05
ratio = n2 / n1

for pow in [0.8, 0.9, 0.95]:
    Peter = Power(effect_size=effect_size, alpha=alpha, power=pow, ratio=ratio)
    print(f'Para una potencia igual a {pow}:')
    print(f'\t El tamaño mínimo de la muestra M1 es {Peter:.0f}.')
    print(f'\t El tamaño mínimo de la muestra M2 es {ratio * Peter:.0f}.')

Respecto a lo de utilizar esta información en un juicio por discriminación, pensamos en usar la misma función que antes, pero esta vez para determinar la potencia del test, fijando los tamaños de las muestras. Vemos que la potencia resultante es igual a 1 (hasta el 76 orden de magnitud). Esto nos lleva a creer (con un 100% de muchísima confianza) que tenemos todas las de ganar.


In [ ]:
effect_size = (m1 - m2) / s2
alpha = 0.05
ratio = n2 / n1
nobs1 = n1

Peter = Power(effect_size=effect_size, alpha=alpha, nobs1=nobs1, ratio=ratio)
print(f'\t Potencia del test con nuestros tamaños muestrales:')
print(f'{Peter:.76f}')

---
# Ejercicio 3 - Comunicación y visualización

Para responder este ejercicio elegimos hacer un tweet. El mismo sería:

> **Febrero/2022: los varones cis no sólo ocupan el 82% de los puestos de trabajo IT, sino que además cobran entre un 8 y un 15% más que el resto de los géneros. INDIGNANTE. #injusticia #TECHodecristal #patriarcado**

La imagen que acompaña está a continuación.

Los datos fueron tomados de la encuesta de sysarmy, ya utilizada a lo largo del entregable.

In [ ]:
print(f'Proporción de varones cis en IT: {n1/(n1+n2):.2f}.')

fig, ax = plt.subplots(figsize=(6,5))

sns.stripplot(data=df1, x='genero', y='neto', hue='genero', size=5, alpha=0.5, ax=ax, legend=False)

ax.set_ylabel('Salario mensual neto ($)')
ax.set_xlabel('Género')
ax.set_yticklabels(['0','100k', '150k', '200k', '250k', '300k', '350k', '400k', '450k', '500k'])
ax.set_xticklabels(['Varón cis','Otros'])

ax.grid(visible=True, which='major', axis='y')
ax.hlines(m1, -0.5, 1.5, color="tab:blue", label = f'Promedio = $ 228k', ls='-', zorder=5)
ax.hlines(m2, -0.5, 1.5, color="tab:orange", label = f'Promedio = $ 204k', ls='-', zorder=5)
ax.legend(loc='lower center', fontsize=9)

plt.show()